In [1]:
import pandas as pd
import numpy as np


In [2]:
credits=pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')

In [3]:
print(movies.shape)

(4803, 20)


In [4]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [5]:
print(credits.shape)
credits.head()

(4803, 4)


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
credits.isnull().sum()

movie_id    0
title       0
cast        0
crew        0
dtype: int64

In [7]:
credits=credits[['movie_id','cast','crew']]

In [8]:
credits.rename(columns={'movie_id':'id'},inplace=True)
credits.head()

,id,cast,crew
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [9]:
df=movies.merge(credits,on='id')

In [10]:
df.sample(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
4723,100000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",http://thegallowsmovie.com/,299245,"[{""id"": 2125, ""name"": ""gallows""}, {""id"": 6270,...",en,The Gallows,20 years after a horrific accident during a sm...,18.045782,"[{""name"": ""New Line Cinema"", ""id"": 12}, {""name...",...,42664410,87.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Every School Has Its Spirit,The Gallows,4.9,361,"[{""cast_id"": 0, ""character"": ""Cassidy"", ""credi...","[{""credit_id"": ""54951e769251413af7001c60"", ""de..."


In [11]:
# General IMDB wala
# Content based system

In [12]:
# weighted rating (WR) = [(v/(v+m)) × R] + [(m/(v+m)) × C]

# Where:

# R = average for the movie (mean) = (rating)

# v = number of votes for the movie = (votes)

# m = minimum votes required to be listed in the Top Rated list (currently 25,000)

# C = the mean vote across the whole report

In [13]:
m=25000
C=df['vote_average'].mean()

In [14]:
def weighted_rating(row,m=m,C=C):
    
    R=row['vote_average']
    v=row['vote_count']
    
    return ((v/(v+m))*R) + ((m/(v+m))*C)

In [15]:
df['wr']=df.apply(weighted_rating,axis=1)

In [16]:
df[['title','wr']].sort_values('wr',ascending=False)

,title,wr
96,Inception,6.804694
65,The Dark Knight,6.775869
95,Interstellar,6.700504
662,Fight Club,6.696080
1881,The Shawshank Redemption,6.687149
...,...,...
210,Batman & Robin,5.990608
1154,Fifty Shades of Grey,5.989421
91,Independence Day: Resurgence,5.984147
337,A Good Day to Die Hard,5.982799


In [17]:
# Important cols for content based filtering

# 1. Genre
# 2. keywords
# 3. overview
# 4. cast
# 5. crew

In [18]:
df['original_language'].value_counts()

en    4505
fr      70
es      32
de      27
zh      27
hi      19
ja      16
it      14
cn      12
ko      11
ru      11
pt       9
da       7
sv       5
fa       4
nl       4
he       3
th       3
ta       2
cs       2
ar       2
ro       2
id       2
no       1
is       1
pl       1
vi       1
te       1
af       1
el       1
ky       1
xx       1
sl       1
ps       1
hu       1
nb       1
tr       1
Name: original_language, dtype: int64

In [19]:
def recommend(movie_name):
    pass

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(stop_words='english')

In [21]:
df['overview'].fillna(' ',inplace=True)

In [22]:
overview_matrix=cv.fit_transform(df['overview']).toarray()

In [23]:
overview_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [24]:
def recommend(movie_name):
    # find the index of this movie
    index_pos=df[df['title']==movie_name].index[0]
    
    # calulate similarity
    recommended_movie_index=sorted(list(enumerate(similarity_score[index_pos])),reverse=True,key=lambda x:x[1])[1:6]
    
    # movie name from index
    
    for i in recommended_movie_index:
        
        print(df.iloc[i[0]].title)
    

In [28]:
recommend('Harry Potter and the Prisoner of Azkaban')

Harry Potter and the Goblet of Fire
Harry Potter and the Chamber of Secrets
Harry Potter and the Order of the Phoenix
Goldfinger
Pocahontas


In [26]:
from sklearn.metrics.pairwise import cosine_similarity


In [27]:
similarity_score=cosine_similarity(overview_matrix)

In [29]:
similarity_score

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.03984095, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.02981424, 0.        ,
        0.        ],
       ...,
       [0.        , 0.03984095, 0.02981424, ..., 1.        , 0.04348854,
        0.02328101],
       [0.        , 0.        , 0.        , ..., 0.04348854, 1.        ,
        0.04556057],
       [0.        , 0.        , 0.        , ..., 0.02328101, 0.04556057,
        1.        ]])

In [30]:
sorted(list(enumerate(similarity_score[200])),reverse=True,key=lambda x:x[1])[1:11]

[(183, 0.23836564731139806),
 (102, 0.1417366773784602),
 (2306, 0.11180339887498948),
 (2973, 0.10206207261596575),
 (4438, 0.10206207261596575),
 (4141, 0.0944911182523068),
 (1760, 0.08333333333333333),
 (426, 0.08164965809277258),
 (1300, 0.07715167498104596),
 (14, 0.0753778361444409)]

In [31]:
df.iloc[15].title

'The Chronicles of Narnia: Prince Caspian'

In [32]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew', 'wr'],
      dtype='object')

In [33]:
df[['title','cast','crew','genres','keywords']]['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [34]:
['Action','Adventure','Science Fiction']

['Action', 'Adventure', 'Science Fiction']

In [35]:
# genres
from ast import literal_eval

In [36]:
def clean_genre(x):
    genre=[]
    for i in literal_eval(x):
        genre.append(i['name'])
    return genre

In [37]:
df['genres']=df['genres'].apply(clean_genre)

In [89]:
df[['title','cast','crew','genres','keywords']].shape

(4629, 5)

In [39]:
def clean_keyword(x):
    keywords=[]
    for i in literal_eval(x):
        keywords.append(i['name'])
    return keywords

In [40]:
df['keywords']=df['keywords'].apply(clean_keyword)

In [41]:
df['keywords']

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4798    [united states–mexico barrier, legs, arms, pap...
4799                                                   []
4800    [date, love at first sight, narration, investi...
4801                                                   []
4802            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4803, dtype: object

In [42]:
def clean_crew(x):
    for i in literal_eval(x):
        if i['job']=='Director':
            return i['name']
        

In [43]:
df['crew']=df['crew'].apply(clean_crew)

In [44]:
df['crew']

0           James Cameron
1          Gore Verbinski
2              Sam Mendes
3       Christopher Nolan
4          Andrew Stanton
              ...        
4798     Robert Rodriguez
4799         Edward Burns
4800          Scott Smith
4801          Daniel Hsia
4802     Brian Herzlinger
Name: crew, Length: 4803, dtype: object

In [45]:
def clean_cast(x):
    actor=[]
    count=0
    for i in literal_eval(x):
        if count<4:
            actor.append(i['name'])
            count+=1
        else:
            return actor
    

In [46]:
df['cast']=df['cast'].apply(clean_cast)

In [47]:
df['crew'][0]

'James Cameron'

In [48]:
def clean(x):
    
    y=[]
    
    if type(x)==list:
        
        for i in x:
            
            i=i.lower()
            i=i.replace(" ","")
            y.append(i)
            
        return y
    if type(x)==str:
        
        x=x.lower()
        x=x.replace(" ","")
        
        return x

In [49]:
features=['cast','crew','genres','keywords']

for i in features:
    
    df[i]=df[i].apply(clean)
    

In [88]:
df[['id','cast','genres','crew','keywords']]

,id,cast,genres,crew,keywords
0,19995,"[samworthington, zoesaldana, sigourneyweaver, ...","[action, adventure, fantasy, sciencefiction]",jamescameron,"[cultureclash, future, spacewar, spacecolony, ..."
1,285,"[johnnydepp, orlandobloom, keiraknightley, ste...","[adventure, fantasy, action]",goreverbinski,"[ocean, drugabuse, exoticisland, eastindiatrad..."
2,206647,"[danielcraig, christophwaltz, léaseydoux, ralp...","[action, adventure, crime]",sammendes,"[spy, basedonnovel, secretagent, sequel, mi6, ..."
3,49026,"[christianbale, michaelcaine, garyoldman, anne...","[action, crime, drama, thriller]",christophernolan,"[dccomics, crimefighter, terrorist, secretiden..."
4,49529,"[taylorkitsch, lynncollins, samanthamorton, wi...","[action, adventure, sciencefiction]",andrewstanton,"[basedonnovel, mars, medallion, spacetravel, p..."
...,...,...,...,...,...
4798,9367,"[carlosgallardo, jaimedehoyos, petermarquardt,...","[action, crime, thriller]",robertrodriguez,"[unitedstates–mexicobarrier, legs, arms, paper..."
4799,72766,"[edwardburns, kerrybishé, marshadietlein, cait...","[comedy, romance]",edwardburns,[]
4800,231617,"[ericmabius, kristinbooth, crystallowe, geoffg...","[comedy, drama, romance, tvmovie]",scottsmith,"[date, loveatfirstsight, narration, investigat..."
4801,126186,"[danielhenney, elizacoupe, billpaxton, alanruck]",[],danielhsia,[]


In [51]:
def merge_cols(x):
    y=""
    
    y = y + x['crew'] + " " + " ".join(x['cast']) + " " + " ".join(x['genres']) + " " + " ".join(x['keywords'])
    
    return y

In [52]:
df.dropna(subset=['cast','crew'],inplace=True)

In [53]:
df['all']=df.apply(merge_cols,axis=1)

In [54]:
cv=CountVectorizer(stop_words='english')

In [55]:
movie_matrix=cv.fit_transform(df['all']).toarray()

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
similarity_matrix=cosine_similarity(movie_matrix)

In [65]:
def recommend(movie_name):
    # find the index of this movie
    index_pos=df[df['title']==movie_name].index[0]
    print(index_pos)
    
    # calulate similarity
    recommended_movie_index=sorted(list(enumerate(similarity_matrix[index_pos])),reverse=True,key=lambda x:x[1])[1:11]
    
    # movie name from index
    
    for i in recommended_movie_index:
        print(df.iloc[i[0]].title)
    

In [91]:
recommend("Titanic")

25
The Great Gatsby
Revolutionary Road
Trash
Love Letters
The Time Traveler's Wife
A Walk to Remember
Angel Eyes
Pompeii
The Illusionist
Romeo + Juliet


In [94]:
df[['id','title','all']].to_csv('movie.csv')

In [95]:
from flask import Flask

In [ ]:
clf = Flask()